In [1]:
# -*- coding: utf-8 -*-

"""
This program makes learning ev-gmm.
"""

'\nThis program makes learning ev-gmm.\n'

In [2]:
# __future__ module make compatible python2 and python3
from __future__ import division, print_function

# basic modules
import os
import os.path
import time

# for warning ignore
import warnings
#warning.filterwarnings('ignore')

# for file system manupulation 
from shutil import rmtree 
import glob
import argparse

# for save object
import pickle

# for make glaph
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (16, 5)
import librosa.display

# for scientific computing
import numpy as np
from numpy.linalg import norm 
from sklearn.decomposition import PCA
from sklearn.mixture import GMM # GMM class cannot use after sklearn 0.20.0
from sklearn.preprocessing import StandardScaler
import scipy.sparse
from scipy.signal import firwin, lfilter

# for display audio controler
from IPython.display import Audio

# for manuplate audio data
import soundfile as sf
import pyworld as pw
import pysptk
from dtw import dtw
from fastdtw import fastdtw

In [3]:
class WORLD(object):
    """
    WORLD based speech analyzer and synthezer.
    
    Ref : https://github.com/k2kobayashi/sprocket/
    """
    def __init__(self, fs=16000, fftl=1024, shiftms=5.0, minf0=40.0, maxf0=500.0):
        """
        Parameters
        ----------
        fs : int 
            Sampling frequency 
        fftl : int
            FFT length
        shiftms : float
            Shift length [ms]
        minf0 : float
            Floor in F0 estimation
        maxf0 : float
            Seli in F0 estimation
        """
        
        self.fs = fs
        self.fftl = fftl
        self.shiftms = shiftms
        self.minf0 = minf0
        self.maxf0 = maxf0
        
    def analyze(self, x):
        """
        Analyze acoustic featueres.
        
        Parameters
        ----------
        x : array, shape(`T`)
            monoral speech signal in time domain
        
        Returns
        ----------
        f0 : array, shape(`T`)
            F0 sequence
        sp : array, shape(`T`, `fftl / 2 + 1`)
            Spectral envelope sequence
        ap : array, shape(`T`, `fftl / 2 + 1`)
            aperiodicity sequence
        """
        
        f0, time_axis = pw.harvest(x, self.fs, f0_floor=self.minf0,
                                   f0_ceil=self.maxf0, frame_period=self.shiftms)
        sp = pw.cheaptrick(x, f0, time_axis, self.fs, fft_size=self.fftl)
        ap = pw.d4c(x, f0, time_axis, self.fs, fft_size=self.fftl)
        
        assert sp.shape == ap.shape
        
        return f0, sp, ap
    
    def analyze_f0(self, x):
        """
        Analyze f0.
        
        Parameters
        ----------
        x : array, shape(`T`)
            monoral speech signal in time domain
        
        Returns
        ----------
        f0 : array, shape(`T`)
            F0 sequence
        """

        f0, time_axis = pw.harvest(x, self.fs, f0_floor=self.minf0,
                                   f0_ceil=self.maxf0, frame_period=self.shiftms)
        
        assert f0.shape == x.shape()
        
        return f0
    
    def synthesis(self, f0, sp, ap):
        """
        Re-synthesizes a speech waveform from acoustic featueres.
        
        Parameters
        ----------
        f0 : array, shape(`T`)
            F0 sequence
        sp : array, shape(`T`, `fftl / 2 + 1`)
            Spectral envelope sequence
        ap : array, shape(`T`, `fftl / 2 + 1`)
            aperiodicity sequence
        """

        return pw.synthesize(f0, sp, ap, self.fs, frame_period=self.shiftms)

In [4]:
class FeatureExtractor(object):
    """
    Analyze acoustic features from a waveform.
    
    This class may have several types of estimeter like WORLD or STRAIGHT.
    Default type is WORLD.
    
    Ref : https://github.com/k2kobayashi/sprocket/
    """
    
    def __init__(self, analyzer='world', fs=16000, fftl=1024, 
                 shiftms=5.0, minf0=50.0, maxf0=500.0):
        """
        Parameters
        ----------
        analyzer : str
            Analyzer
        fs : int 
            Sampling frequency 
        fftl : int
            FFT length
        shiftms : float
            Shift length [ms]
        minf0 : float
            Floor in F0 estimation
        maxf0 : float
            Seli in F0 estimation
        """
        
        self.analyzer = analyzer
        self.fs = fs
        self.fftl = fftl
        self.shiftms = shiftms
        self.minf0 = minf0
        self.maxf0 = maxf0
    
        if self.analyzer == 'world':
            self.analyzer = WORLD(fs=self.fs, fftl=self.fftl, 
                                  minf0=self.minf0, maxf0=self.maxf0, shiftms=self.shiftms)
        else:
            raise('Analyzer Error : not support type, see FeatureExtractor class.')
        
        self._f0 = None
        self._sp = None
        self._ap = None
        
    def analyze(self, x):
        """
        Analyze acoustic featueres.
        
        Parameters
        ----------
        x : array, shape(`T`)
            monoral speech signal in time domain
        
        Returns
        ----------
        f0 : array, shape(`T`)
            F0 sequence
        sp : array, shape(`T`, `fftl / 2 + 1`)
            Spectral envelope sequence
        ap : array, shape(`T`, `fftl / 2 + 1`)
            aperiodicity sequence
        """
        
        self.x = np.array(x, dtype=np.float)
        self._f0, self._sp, self._ap = self.analyzer.analyze(self.x)
        
        # check f0 < 0
        self._f0[self._f0 < 0] = 0
        
        if np.sum(self._f0) == 0.0:
            print("Warning : F0 values are all zero.")
        
        return self._f0, self._sp, self._ap
    
    def analyze_f0(self, x):
        """
        Analyze f0.
        
        Parameters
        ----------
        x : array, shape(`T`)
            monoral speech signal in time domain
        
        Returns
        ----------
        f0 : array, shape(`T`)
            F0 sequence
        """

        self.x = np.array(x, dtype=np.float)
        self._f0 = self.analyzer.analyze_f0(self.x)

        # check f0 < 0
        self._f0[self._f0 < 0] = 0
        
        if np.sum(self._f0) == 0.0:
            print("Warning : F0 values are all zero.")
        
        return self._f0
    
    def mcep(self, dim=24, alpha=0.42):
        """
        Convert mel-cepstrum sequence from spectral envelope.
        
        Parameters
        ----------
        dim : int
            mel-cepstrum dimension
        alpha : float
            parameter of all-path filter
        
        Returns
        ----------
        mcep : array, shape(`T`, `dim + 1`)
            mel-cepstrum sequence
        """        
        
        self._analyzed_check()
        
        return pysptk.sp2mc(self._sp, dim, alpha)
    
    def codeap(self):
        """
        """
        self._analyzed_check()
        
        return pw.code_aperiodicity(self._ap, self.fs)
    
    def npow(self):
        """
        Normalized power sequence from spectral envelope.
        
        Returns
        ----------
        npow : vector, shape(`T`, `1`)
            Normalized power sequence of the given waveform
        """
        
        self._analyzed_check()
        
        npow = np.apply_along_axis(self._spvec2pow, 1, self._sp)
        
        meanpow = np.mean(npow)
        npow = 10.0 * np.log10(npow / meanpow)
        
        return npow
    
    def _spvec2pow(self, specvec):
        """
        """
        fftl2 = len(specvec) - 1
        fftl = fftl2 * 2
        
        power = specvec[0] + specvec[fftl2]
        for k in range(1, fftl2):
            power += 2.0 * specvec[k]
        power /= fftl
        
        return power
        
    def _analyzed_check(self):
        if self._f0 is None and self._sp is None and self._ap is None:
            raise('Call FeatureExtractor.analyze() before this method.')

In [5]:
class Synthesizer(object):
    """
    Synthesize a waveform from acoustic features.
    
    Ref : https://github.com/k2kobayashi/sprocket/
    """
    def __init__(self, fs=16000, fftl=1024, shiftms=5.0):
        """
        Parameters
        ----------
        fs : int 
            Sampling frequency 
        fftl : int
            FFT length
        shiftms : float
            Shift length [ms]
        """
        
        self.fs = fs
        self.fftl = fftl
        self.shiftms = shiftms
    
    def synthesis(self, f0, mcep, ap, rmcep=None, alpha=0.42):
        """
        Re-synthesizes a speech waveform from acoustic featueres.
        
        Parameters
        ----------
        f0 : array, shape(`T`)
            F0 sequence
        mcep : array, shape(`T`, `dim`)
            mel-cepstrum sequence
        ap : array, shape(`T`, `fftl / 2 + 1`)
            aperiodicity sequence
        rmcep : array, shape(`T`, `dim`)
            array of reference mel-cepstrum sequence
        alpha : float
            parameter of all-path filter
            
        Returns
        ----------
        wav : array,
            syntesized waveform
        """
        
        if rmcep is not None:
            # power modification
            mcep = mod_power(mcep, rmcep, alpha=alpha)
        
        sp = pysptk.mc2sp(mcep, alpha, self.fftl)
        wav = pw.synthesize(f0, sp, ap, self.fs, frame_period=self.shiftms)
        
        return wav
        
    def synthesis_diff(self, x, diffmcep, rmcep=None, alpha=0.42):
        """
        Re-synthesizes a speech waveform from acoustic featueres.
        filtering with a differential mel-cepstrum.
        
        Parameters
        ----------
        x : array, shape(`samples`)
            array of waveform sequence
        diffmcep : array, shape(`T`, `dim`)
            array of differential mel-cepstrum sequence
        rmcep : array, shape(`T`, `dim`)
            array of reference mel-cepstrum sequence
        alpha : float
            parameter of all-path filter
            
        Returns
        ----------
        wav : array,
            syntesized waveform
        """      
        
        x = x.astype(np.float64)
        dim = diffmcep.shape[1] - 1 
        shiftl = int(self.fs / 1000 * self.shiftms)
        
        if rmcep is not None:
            # power modification
            diffmcep = mod_power(rmcep + diffmcep, rmcep, alpha=alpha) - rmcep        
        
        # mc2b = transform mel-cepstrum to MLSA digital filter coefficients.
        b = np.apply_along_axis(pysptk.mc2b, 1, diffmcep, alpha)
        
        mlsa_fil = pysptk.synthesis.Synthesizer(pysptk.synthesis.MLSADF(dim, alpha=alpha),
                                                shiftl)
        wav = mlsa_fil.synthesis(x, b)
        
        return wav
    
    def synthesis_sp(self, f0, sp, ap):
        """
        Re-synthesizes a speech waveform from acoustic featueres.
        
        Parameters
        ----------
        f0 : array, shape(`T`)
            F0 sequence
        spc : array, shape(`T`, `dim`)
            mel-cepstrum sequence
        ap : array, shape(`T`, `fftl / 2 + 1`)
            aperiodicity sequence
            
        Returns
        ----------
        wav : array,
            syntesized waveform
        """      
        
        wav = pw.synthesize(f0, sp, ap, self.fs, frame_period=self.shiftms)
        
        return wav
    
    def mod_power(cvmcep, rmcep, alpha=0.42, irlen=256):
        """
        power modification based on inpuulse responce
        
        Parameters
        ----------
        cvmcep : array, shape(`T`, `dim`)
            array of converted mel-cepstrum
        rmcep : arraym shape(`T`, `dim`)
            array of reference mel-cepstrum
        alpha : float
            parameter of all-path filter
        irlen : int
            Length for IIR filter
        
        Returns
        ----------
        modified_cvmcep : array, shape(`T`, `dim`)
            array of power modified converted mel-cepstrum
        """
        
        if rmcep.shape != cvmcep.shape:
            raise ValueError(
                "The shape of the converted and reference mel-cepstrum are different : {} / {}.format(cvmcep.shape, rmcep.shape)"
            )
        
        # mc2e = Compute energy from mel-cepstrum. e-option
        cv_e = pysptk.mc2e(cvmcep, alpha=alpha, irlen=irlen)
        r_e = pysptk.mc2e(rmcep, alpha=alpha, irlen=irlen)
        
        dpow = np.log(r_e / cv_e) / 2
        
        modified_cvmcep = np.copy(cvmcep)
        modified_cvmcep[:, 0] += dpow
        
        return modified_cvmcep

In [6]:
# def util methods
def melcd(array1, array2):
    """
    calculate mel-cepstrum distortion
    
    Parameters
    ----------
    array1, array2 : array, shape(`T`, `dim`) or shape(`dim`)
        Array of original and target.
    
    Returns
    ----------
    mcd : scala, number > 0
        Scala of mel-cepstrum distoriton
    """
    if array1.shape != array2.shape:
        raise ValueError(
            "The shape of both array are different : {} / {}.format(array1.shape,array2.shape)"
        )    
   
    if array1.ndim == 2:
        diff = array1 - array2
        mcd = 10.0 / np.log(10) * np.mean(np.sqrt(2.0 * np.sum(diff ** 2, axis=1)))
    elif array1.ndim == 1:
        diff = array1 - array2
        mcd = 10.0 / np.log(10) * np.sqrt(2.0 * np.sum(diff ** 2))
    else:
        raise ValueError("Dimension mismatch.")
        
    return mcd

def delta(data, win=[-1.0, 1.0, 0]):
    """
    calculate delta component
    
    Parameters
    ----------
    data : array, shape(`T`, `dim`)
        Array of static matrix sequence.
    win : array, shape(`3`)
        The shape of window matrix.
    
    Returns
    ----------
    delta : array, shape(`T`, `dim`)
        Array of delta matrix sequence.
    """
    
    if data.dim == 1:
        # change vector into 1d-array
        T = len(data)
        dim = data.ndim
        data = data.reshape(T, dim)
    else:
        T, dim = data.shape
    
    win = np.array(win, dtype=np.float64)
    delta = np.zeros((T, dim))
    
    delta[0] = win[0] * data[0] + win[1] * data[1]
    delta[-1] = win[0] * data[-2] + win[1] * data[-1]
    
    for i in range(len(win)):
        delta[1:T - 1] += win[i] * delta[i:T - 2 + i]
    
    return delta

def static_delta(data, win=[-1.0, 1.0, 0]):
    """
    calculate static and delta component
    
    Parameters
    ----------
    data : array, shape(`T`, `dim`)
        Array of static matrix sequence.
    win : array, shape(`3`)
        The shape of window matrix.
    
    Returns
    ----------
    sddata : array, shape(`T`, `dim * 2`)
        Array of static and delta matrix sequence.
    """
    
    sddata = np.c_[data, delta(data, win)]
    
    assert sddata.shape[1] == data.shape[1] * 2
    
    return sddata

def construct_static_and_delta_matrix(T, D, win=[-1.0, 1.0, 0]):
    """
    calculate static and delta transformation matrix
    
    Parameters
    ----------
    T : scala, `T`
        Scala of time length
    D : scala, `D`
        Scala of the number of dimension.
    win : array, shape(`3`)
        The shape of window matrix.
    
    Returns
    ----------
    W : array, shape(`2 * D * T`, `D * T`)
        Array of static and delta transformation matrix.
    """
    
    static = [0, 1, 0]
    delta = win
    assert len(static) == len(delta)
    
    # generate full W
    DT = D * T
    ones = np.ones(DT)
    row = np.arange(2 * DT).reshape(2 * T, D) # generate serial numbers
    static_row = row[::2] # [1,2,3,4,5] => [1,3,5]
    delta_row = row[1::2] # [1,2,3,4,5] => [2,4]
    col = np.arange(DT)
    
    delta = np.array([ones * static[0], ones * static[1],
                      ones * static[2], ones * delta[0],
                      ones * delta[1], ones * delta[2]]).flatten()
    row = np.array([[static_row] * 3, [delta_row] * 3]).flatten()
    col = np.array([[col - D, col, col + D] * 2]).flatten()

    # remove component at first and end frame
    valid_idx = np.logical_not(np.logical_or(col < 0, col >= DT))
    
    W = scipy.sparse.csr_matrix(
        (data[valid_idx], (row[valid_idx], col[valid_idx])), shape=(2 * DT, DT))
    W.eliminata_zeros()
    
    return W
    
def extfrm(data, npow, threshold=-20):
    """
    Extract frame over the power threshold
    
    Parameters
    ----------
    data : array, shape(`T`, `dim`)
        array of input data
    npow : array, shape(`T`)
        vector of normalized power sequence
    threshold : scala
        scala of power threshold [dB]
        
    Returns
    ----------
    data : array, shape(`T_ext`, `dim`)
        remaining data after extracting frame
        `T_ext` <= `T`
    """
    T = data.shape[0]
    if T != len(npow):
        raise("Length of two vectors is different.")
        
    valid_index = np.where(npow > threshold)
    
    return data[valid_index]

def estimate_twf(orgdata, tardata, distance='melcd', fast=True):
    """
    time warping function estimator
    
    Parameters
    ----------
    orgdata : array, shape(`T_org`, `dim`)
        array of source feature
    tardata : array, shape(`T_tar`, `dim`)
        array of target feature
    distance : str 
        distance function
    fast : bool
        use fastdtw instead of dtw
    
    Returns
    ----------
    twf : array, shape(`2`, `T`)
        time warping function between original and target
    """
    
    if distance == 'melcd':
        def distance_func(x, y): return melcd(x, y)
    else:
        raise ValueError('this distance method is not support.')
    
    if fast:
        _, path = fastdtw(orgdata, tardata, dist=distance_func)
        twf = np.array(path).T
    else:
        _, _, _, twf = dtw(orgdata, tardata, distance_func)
    
    return twf

def low_cut_filter(x, fs, cutoff=70):
    """
    low cut filter
    
    Parameters
    ----------
    x : array, shape('samples')
        waveform sequence
    fs : array, int
        Sampling frequency
    cutoff : float
        cutoff frequency of low cut filter
    
    Returns
    ----------
    lct_x : array, shape('samples')
        Low cut filtered waveform sequence
    """
    
    nyquist = fs // 2
    norm_cutoff = cutoff / nyquist
    
    # low cut filter
    fil = firwin(255, norm_cutoff, pass_zero=False)
    lct_x = lfilter(fil, 1, x)
    
    return lct_x

In [7]:
class F0statistics(object):
    """
    Estimate F0 statistics and convert F0
    """
    def __init__(self):
        pass
    
    def estimate(self, f0list):
        """
        estimate F0 statistics from list of f0
        
        Parameters
        ----------
        f0list : list, shape(`f0num`)
            List of several F0 sequence
        
        Returns
        ----------
        f0stats : array, shape(`[mean, std]`)
            values of mean and standard deviation for log f0
        """
        
        n_files = len(f0list)
        for i in range(n_files):
            f0 = f0list[i]
            nonzero_indices = np.nonzero(f0)
            if i == 0:
                f0s = np.log(f0[nonzero_indices])
            else:
                f0s = np.r_[f0s, np.log(f0[nonzero_indices])]
        
        f0stats = np.array([np.mean(f0s), np.std(f0s)])
        
        return f0stats

    def convert(self, f0, orgf0stats, tarf0stats):
        """
        convert F0 based on F0 statistics
        
        Parameters
        ----------
        f0 : array, shape(`T`, `1`)
            array of F0 sequence
        orgf0stats : array, shape(`[mean, std]`)
            vectors of mean and standard deviation of log f0 for original speaker
        tarf0stats : array, shape(`[mean, std]`)
            vectors of mean and standard deviation of log f0 for target speaker
        
        Returns
        ----------
        cvf0 : array, shape(`T`, `1`)
            array of converted F0 sequence
        """
        
        # get length and dimension
        T = len(f0)
        
        # perform f0 conversion
        cvf0 = np.zeros(T)
        
        nonzero_indices  = f0 > 0
        cvf0[nonzero_indices] = np.exp((tarf0stats[1] / orgf0stats[1]) 
                                       * (np.log(f0[nonzero_indices])) 
                                       - orgf0stats[0] + tarf0stats[0])
        
        return cvf0

In [ ]:
class GV(object):
    """
    Estimate statistics and perform postfilter based on the GV statistics.
    """
    def __init__(self):
        pass
    
    def estimate(self, datalist):
        """
        estimate GV statistics from list of data
        
        Parameters
        ----------
        datalist : list, shape(`num_data`)
            List of several data ([T, dim]) sequence
        
        Returns
        ----------
        gvstats : array, shape(`2`, `dim`)
            array of mean and standard deviation for GV
        """
        
        n_files = len(datalist)
        
        var = []
        for i in range(n_files):
            data = datalist[i]
            var.append(np.var(data, axis=0))
            
        # calculate vm and vv
        vm = np.mean(np.array(var), axis=0)
        vv = np.var(np.array(var), axis=0)
        gvstats = np.r_[vm, vv]
        gvstats = gvstats.reshape(2, len(vm))
        
        return gvstats

    def postfilter(self, data, gvstats, cvgvstats=None, alpha=1.0, startdim=1):
        """
        perform postfilter based on GV statistics into data
        
        Parameters
        ----------
        data : array, shape(`T`, `dim`)
            array of data sequence
        gvstats : array, shape(`2`, `dim`)
            array of mean and variance for target GV
        cvgvstats : array, shape(`2`, `dim`)
            array of mean and variance for converted GV
        alpha : float
            morphing coefficient between GV transformed data and data.
            alpha * gvpf(data) + (1 - alpha) * data
        startdim : int
            start dimension to perform GV postfilter
        
        Returns
        ----------
        filtered_data : array, shape(`T`, `data`)
            array of GV postfiltered data sequnece
        """
        
        # get length and dimension
        T, dim = data.shape
        assert gvstats is not None
        assert dim == gvstats.shape[1]
        
        # calculate statics of input data
        datamean = np.mean(data, axis=0)
        
        if cvgvstats is None:
            # use variance of the given data
            datavar = np.var(data, axis=0)
        else:
            # use variance of trained gv stats
            datavar = cvgvstats[0]
        
        # perform GV postfilter
        filterd = np.sqrt(gvstats[0, startdim:] / datavar[startdim:]) * (data[:, startdim:] - datamean[startdim:]) + datamean[startdim:]
        
        filterd_data = np.c_[data[:, :startdim], filterd]
        
        return alpha * filterd_data + (1 - alpha) * data

In [ ]:
# 0. config path
__versions = "pre-stored-ej"
__same_path = "./utterance/" + __versions + "/"
pre_stored_source_list = __same_path + 'pre-source/**/V01/T01/**/*.wav'
pre_stored_list = __same_path + "pre/**/V01/T01/**/*.wav"
output_path = __same_path + "output/"

# 1. estimate features
feat = FeatureExtractor()
synthesizer = Synthesizer()

org_f0list = None
org_splist = None
org_mceplist = None
org_aplist = None
org_npowlist = None
org_codeaplist = None

if os.path.exists(output_path + "_org_f0.pickle") \
    and os.path.exists(output_path + "_org_sp.pickle") \
    and os.path.exists(output_path + "_org_ap.pickle") \
    and os.path.exists(output_path + "_org_mcep.pickle") \
    and os.path.exists(output_path + "_org_npow.pickle") \
    and os.path.exists(output_path + "_org_codeap.pickle"):
        
    with open(output_path + "_org_f0.pickle", 'rb') as f:    
        org_f0list = pickle.load(f)
    with open(output_path + "_org_sp.pickle", 'rb') as f:   
        org_mceplist = pickle.load(f)
    with open(output_path + "_org_ap.pickle", 'rb') as f:  
        org_aplist = pickle.load(f)
    with open(output_path + "_org_mcep.pickle", 'rb') as f:  
        org_npowlist = pickle.load(f)
    with open(output_path + "_org_npow.pickle", 'rb') as f:  
        org_npowlist = pickle.load(f)
    with open(output_path + "_org_codeap.pickle", 'rb') as f:  
        org_codeaplist = pickle.load(f) 
else:
    org_f0list = []
    org_splist = []
    org_mceplist = []
    org_aplist = []
    org_npowlist = []
    org_codeaplist = []
    for files in sorted(glob.iglob(pre_stored_source_list, recursive=True)):
        wavf = files
        x, fs = sf.read(wavf)
        x = np.array(x, dtype=np.float)
        x = low_cut_filter(x, fs, cutoff=70)
        assert fs == 16000

        print("extract acoustic featuers: " + wavf)

        f0, sp, ap = feat.analyze(x)
        mcep = feat.mcep()
        npow = feat.npow()
        codeap = feat.codeap()
        #name, ext = os.path.splitext(wavf)
        #np.save(name + "_or_f0", f0)
        #np.save(name + "_or_sp", sp)
        #np.save(name + "_or_ap", ap)
        #np.save(name + "_or_mcep", mcep)
        #np.save(name + "_or_codeap", codeap)
        org_f0list.append(f0)
        org_splist.append(sp)
        org_mceplist.append(mcep)
        org_aplist.append(ap)
        org_npowlist.append(npow)
        org_codeaplist.append(codeap)

        #wav = synthesizer.synthesis(f0, mcep, ap)
        #wav = np.clip(wav, -32768, 32767)
        #sf.write(name + "_ansys.wav", wav, fs)

    with open(output_path + "_org_f0.pickle", 'wb') as f:    
        pickle.dump(org_f0list, f)
    with open(output_path + "_org_sp.pickle", 'wb') as f:   
        pickle.dump(org_splist, f)
    with open(output_path + "_org_npow.pickle", 'wb') as f:   
        pickle.dump(org_npowlist, f)
    with open(output_path + "_org_ap.pickle", 'wb') as f:  
        pickle.dump(org_aplist, f)
    with open(output_path + "_org_mcep.pickle", 'wb') as f:  
        pickle.dump(org_mceplist, f)
    with open(output_path + "_org_codeap.pickle", 'wb') as f:  
        pickle.dump(org_codeaplist, f) 

mid_f0list = None
mid_mceplist = None
mid_aplist = None
mid_npowlist = None
mid_splist = None
mid_codeaplist = None        

if os.path.exists(output_path + "_mid_f0.pickle") \
    and os.path.exists(output_path + "_mid_sp.pickle") \
    and os.path.exists(output_path + "_mid_ap.pickle") \
    and os.path.exists(output_path + "_mid_mcep.pickle") \
    and os.path.exists(output_path + "_mid_npow.pickle") \
    and os.path.exists(output_path + "_mid_codeap.pickle"):
        
    with open(output_path + "_mid_f0.pickle", 'rb') as f:    
        mid_f0list = pickle.load(f)
    with open(output_path + "_mid_sp.pickle", 'rb') as f:   
        mid_mceplist = pickle.load(f)
    with open(output_path + "_mid_ap.pickle", 'rb') as f:  
        mid_aplist = pickle.load(f)
    with open(output_path + "_mid_mcep.pickle", 'rb') as f:  
        mid_npowlist = pickle.load(f)
    with open(output_path + "_mid_npow.pickle", 'rb') as f:  
        mid_npowlist = pickle.load(f)
    with open(output_path + "_mid_codeap.pickle", 'rb') as f:  
        mid_codeaplist = pickle.load(f) 
else:        
    mid_f0list = []
    mid_mceplist = []
    mid_aplist = []
    mid_npowlist = []
    mid_splist = []
    mid_codeaplist = []

    for files in sorted(glob.iglob(pre_stored_list, recursive=True)):
        wavf = files
        x, fs = sf.read(wavf)
        x = np.array(x, dtype=np.float)
        x = low_cut_filter(x, fs, cutoff=70)
        assert fs == 16000

        print("extract acoustic featuers: " + wavf)

        f0, sp, ap = feat.analyze(x)
        mcep = feat.mcep()
        npow = feat.npow()
        codeap = feat.codeap()
        name, ext = os.path.splitext(wavf)
        #np.save(name + "_or_f0", f0)
        #np.save(name + "_or_sp", sp)
        #np.save(name + "_or_ap", ap)
        #np.save(name + "_or_mcep", mcep)
        #np.save(name + "_or_codeap", codeap)
        mid_f0list.append(f0)
        mid_splist.append(sp)
        mid_mceplist.append(mcep)
        mid_aplist.append(ap)
        mid_npowlist.append(npow)
        mid_codeaplist.append(codeap)

        #wav = synthesizer.synthesis(f0, mcep, ap)
        #wav = np.clip(wav, -32768, 32767)
        #sf.write(name + "_ansys.wav", wav, fs)
        
    with open(output_path + "_mid_f0.pickle", 'wb') as f:
        pickle.dump(mid_f0list, f)
    with open(output_path + "_mid_npow.pickle", 'wb') as f:
        pickle.dump(mid_npowlist, f)
    with open(output_path + "_mid_sp.pickle", 'wb') as f:
        pickle.dump(mid_splist, f)
    with open(output_path + "_mid_ap.pickle", 'wb') as f:
        pickle.dump(mid_aplist, f)
    with open(output_path + "_mid_mcep.pickle", 'wb') as f:
        pickle.dump(mid_mceplist, f)
    with open(output_path + "_mid_codeap.pickle", 'wb') as f:
        pickle.dump(mid_codeaplist, f) 

extract acoustic featuers: ./utterance/pre-stored0.1.2/pre-source/EJM10/V01/T01/ESPBOBAMA1/000/A03.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre-source/EJM10/V01/T01/ESPBOBAMA1/000/A04.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre-source/EJM10/V01/T01/ESPBOBAMA1/000/A07.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre-source/EJM10/V01/T01/ESPBOBAMA1/000/A08.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre-source/EJM10/V01/T01/ESPBOBAMA1/000/A09.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre-source/EJM10/V01/T01/ESPBOBAMA1/000/A11.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre-source/EJM10/V01/T01/ESPBOBAMA1/000/A13.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre-source/EJM10/V01/T01/ESPBOBAMA1/000/A14.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre-source/EJM10/V01/T01/ESPBOBAMA1/000/A17.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre-source/EJM10/V

extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF02/V01/T01/TIMIT/000/A09.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF02/V01/T01/TIMIT/000/A10.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF02/V01/T01/TIMIT/000/A12.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF02/V01/T01/TIMIT/000/A13.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF02/V01/T01/TIMIT/000/A15.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF02/V01/T01/TIMIT/000/A16.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF03/V01/T01/ESPBOBAMA1/000/A03.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF03/V01/T01/ESPBOBAMA1/000/A04.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF03/V01/T01/ESPBOBAMA1/000/A07.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF03/V01/T01/ESPBOBAMA1/000/A08.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF03/V01/T

extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF04/V01/T01/TIMIT/000/A06.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF04/V01/T01/TIMIT/000/A07.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF04/V01/T01/TIMIT/000/A08.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF04/V01/T01/TIMIT/000/A09.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF04/V01/T01/TIMIT/000/A10.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF04/V01/T01/TIMIT/000/A12.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF04/V01/T01/TIMIT/000/A13.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF04/V01/T01/TIMIT/000/A15.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF04/V01/T01/TIMIT/000/A16.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF05/V01/T01/ESPBOBAMA1/000/A03.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF05/V01/T01/ESPBOBAMA1/0

extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/TIMIT/000/A03.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/TIMIT/000/A04.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/TIMIT/000/A05.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/TIMIT/000/A06.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/TIMIT/000/A07.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/TIMIT/000/A08.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/TIMIT/000/A09.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/TIMIT/000/A10.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/TIMIT/000/A12.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/TIMIT/000/A13.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF06/V01/T01/TIMIT/000/A15.wav

extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF09/V01/T01/ESUS/000/A39.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF09/V01/T01/TIMIT/000/A01.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF09/V01/T01/TIMIT/000/A02.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF09/V01/T01/TIMIT/000/A03.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF09/V01/T01/TIMIT/000/A04.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF09/V01/T01/TIMIT/000/A05.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF09/V01/T01/TIMIT/000/A06.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF09/V01/T01/TIMIT/000/A07.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF09/V01/T01/TIMIT/000/A08.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF09/V01/T01/TIMIT/000/A09.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJF09/V01/T01/TIMIT/000/A10.wav


...

extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM02/V01/T01/ESUS/000/A11.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM02/V01/T01/ESUS/000/A13.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM02/V01/T01/ESUS/000/A14.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM02/V01/T01/ESUS/000/A16.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM02/V01/T01/ESUS/000/A17.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM02/V01/T01/ESUS/000/A18.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM02/V01/T01/ESUS/000/A25.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM02/V01/T01/ESUS/000/A29.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM02/V01/T01/ESUS/000/A30.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM02/V01/T01/ESUS/000/A39.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM02/V01/T01/TIMIT/000/A01.wav
extract a

extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/ESUS/000/A08.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/ESUS/000/A09.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/ESUS/000/A10.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/ESUS/000/A11.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/ESUS/000/A13.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/ESUS/000/A14.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/ESUS/000/A16.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/ESUS/000/A17.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/ESUS/000/A18.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/ESUS/000/A25.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM06/V01/T01/ESUS/000/A29.wav
extract ac

extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM08/V01/T01/ESUS/000/A05.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM08/V01/T01/ESUS/000/A06.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM08/V01/T01/ESUS/000/A07.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM08/V01/T01/ESUS/000/A08.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM08/V01/T01/ESUS/000/A09.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM08/V01/T01/ESUS/000/A10.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM08/V01/T01/ESUS/000/A11.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM08/V01/T01/ESUS/000/A13.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM08/V01/T01/ESUS/000/A14.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM08/V01/T01/ESUS/000/A16.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM08/V01/T01/ESUS/000/A17.wav
extract ac

extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM101/V01/T01/ESPBOBAMA1/000/A22.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM101/V01/T01/ESUS/000/A02.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM101/V01/T01/ESUS/000/A03.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM101/V01/T01/ESUS/000/A05.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM101/V01/T01/ESUS/000/A06.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM101/V01/T01/ESUS/000/A07.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM101/V01/T01/ESUS/000/A08.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM101/V01/T01/ESUS/000/A09.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM101/V01/T01/ESUS/000/A10.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM101/V01/T01/ESUS/000/A11.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM101/V01/T01/ESUS/000/A

extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM12/V01/T01/ESPBOBAMA1/000/A19.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM12/V01/T01/ESPBOBAMA1/000/A20.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM12/V01/T01/ESPBOBAMA1/000/A21.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM12/V01/T01/ESPBOBAMA1/000/A22.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM12/V01/T01/ESUS/000/A02.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM12/V01/T01/ESUS/000/A03.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM12/V01/T01/ESUS/000/A05.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM12/V01/T01/ESUS/000/A06.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM12/V01/T01/ESUS/000/A07.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM12/V01/T01/ESUS/000/A08.wav
extract acoustic featuers: ./utterance/pre-stored0.1.2/pre/EJM12/V01/T01/ESU

In [91]:
def get_aligned_jointdata(orgdata, orgnpow, tardata, tarnpow, cvdata=None,
                          orgpow_threshold=-20, tarpow_threshold=-20):
    """
    get aligment between features
    
    Parameters
    ----------
    orgdata : array, shape(`T_org`, `dim`)
        Acoustic feature of source speaker
    orgnpow : array, shape(`T_org`)
        Normalized power of source speaker
    tardata : array, shape(`T_tar`, `dim`)
        Acoustic feature of target speaker
    tarnpow : array, shape(`T_tar`)
        Normalized power of target speaker
    cvdata : array, shape(`T_org`, `dim`)
        Converted acoustic feature from source into target
    
    Returns
    ----------
    jdata : array, shape(`T_new`, `dim * 2`)
        Joint feature vector between source and target
    twf : array, shape(`T_new`, `2`)
        Time warping function
    mcd : float,
        Mel-cepstrum distortion between source and target
    """
    
    # extract extsddata
    org_extsddata = extfrm(static_delta(orgdata), orgnpow, power_threshold=orgpow_thresold)
    tar_extsddata = extfrm(static_delta(tardata), tarnpow, power_threshold=tarpow_thresold)
    
    if cvdata is None:
        # calculate twf and mel-cd
        twf = estimate_twf(org_extsddata, tar_extsddata, distance='melcd')
        mcd = melcd(org_extsddata[twf[0]], tar_extsddata[twf[1]])
    else:
        if orgdata.shape != cvdata.shape:
            raise ValueError('Dimension mismatch between orgdata and cvdata.')
        # calculate twf and mel-cd with converted data
        cv_extsddata = extfrm(static_delta(cvdata), orgnpow, power_threshold=orgpow_threshold)
        twf = estimate_twf(cv_extsddata, tar_extsddata, distance='melcd')
        mcd = melcd(cv_extsddata[twf[0]], tar_extsddata[twf[1]])
    
    # concatenate joint feature data into joint feature matrix
    jdata = np.c_[org_extsddata[twf[0]], tar_extsddata[twf[1]]]
    
    return jdata, twf, mcd
    

In [ ]:
# 2. estimate twf and jnt

org_mceps = org_mceplist
org_npows = org_npowlist
mid_mceps = mid_mceplist
mid_npows = mid_npowlist

itnum = 1
sd = 1 # start dimension for alignment of mcep
num_files = len(org_mceps)

# first iteration
for i in range(num_files):
    jdata, _, mcd = get_aligned_jointdata(org_mceps[i][:, sd:], org_npows[i], tar_mceps[i][:, sd:], tar_npows[i])
    print("distortion [db] for {}-th file: {}".format(i+1, mcd))
    if i == 0:
        jnt = jdata
    else:
        jnt = np.r_[jnt, jdata]
itnum += 1






